# 머신러닝 복습


# 1.환경준비

## (1) 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import *

## (2) 데이터로딩

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_2000.csv"
data = pd.read_csv(path)
data['CHURN'] = data['CHURN'].map({'STAY':0, 'LEAVE':1})
data.head()

|	구분	|	변수 명	|	내용	|	type	|	비고	|
|	----	|	----	|	----	|	----	|	----	|
|	**Target**	|	**CHURN**	|	이탈여부	|	범주	| 0,1	|
|	feature	|	INCOME	|	소득수준(달러)	|	숫자	|		|
|	feature	|	OVERAGE	|	월평균 초과사용시간(분)	|	숫자	| |
|	feature	|	LEFTOVER	|	월평균 잔여시간(%)	|	숫자	| 	|
|	feature	|	HOUSE	|	집가격(달러)	|	숫자	|	|
|	feature	|	HANDSET_PRICE	|	휴대폰가격(달러)	|	숫자	|		|
|	feature	|	OVER_15MINS_CALLS_PER_MONTH	|	월평균 장기통화 횟수	|	숫자	| 		|
|	feature	|	AVERAGE_CALL_DURATION	|	평균통화시간(분)	|	숫자	|		|

# 2.데이터 준비

### (1) 데이터 분할

In [ ]:
target = 'CHURN'
x = data.drop(target, axis=1)
y = data.loc[:, target]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2, random_state = 10)

### (2) 스케일링

In [ ]:
# 스케일링1
minmax = MinMaxScaler()

x_train_s1 = minmax.fit_transform(x_train)
x_val_s1 = minmax.transform(x_val)

# 스케일링2
standard = StandardScaler()

x_train_s2 = standard.fit_transform(x_train)
x_val_s2 = standard.transform(x_val)

# df로 변환(필수 아님)
x_train_s1 = pd.DataFrame(x_train_s1, columns=x_train.columns)
x_val_s1 = pd.DataFrame(x_val_s1, columns=x_val.columns)

x_train_s2 = pd.DataFrame(x_train_s2, columns=x_train.columns)
x_val_s2 = pd.DataFrame(x_val_s2, columns=x_val.columns)

In [ ]:
x_train.describe().T

In [ ]:
x_train_s1.describe().T

In [ ]:
x_train_s2.describe().T

In [ ]:
plt.figure(figsize = (12,3))
plt.subplot(1,3,1)
sns.histplot(x = 'OVERAGE', data = x_train, kde = True, bins = 30)
plt.title('Original')

plt.subplot(1,3,2)
sns.histplot(x = 'OVERAGE', data = x_train_s1, kde = True, bins = 30)
plt.title('MinMax Scaling')

plt.subplot(1,3,3)
sns.histplot(x = 'OVERAGE', data = x_train_s2, kde = True, bins = 30)
plt.title('Standard Scaling')

plt.tight_layout()
plt.show()

# 3.모델링
* 머신러닝 모델링 연습을 해보겠습니다.
    * 잘 다루는 알고리즘 2~3개를 이용하여 모델링 수행
    * 하이퍼파라미터 튜닝
    * 가장 성능이 좋은 모델 선정

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### (1) 실험1
* 스케일링 방식에 따라 성능은 달라질까?

In [ ]:
# minmax
model1 = KNeighborsClassifier()
model1.fit(x_train_s1, y_train)
pred1 = model1.predict(x_val_s1)

print(accuracy_score(y_val, pred1))
print(confusion_matrix(y_val, pred1))
print(classification_report(y_val, pred1))

In [ ]:
# standard
model2 = KNeighborsClassifier()
model2.fit(x_train_s2, y_train)
pred2 = model2.predict(x_val_s2)

print(accuracy_score(y_val, pred2))
print(confusion_matrix(y_val, pred2))
print(classification_report(y_val, pred2))

### (2) 실험2

* 거리측정 방식에 따라 성능은 달라질까?

In [ ]:
params = {'n_neighbors':range(1,201,2), 'metric':['euclidean', 'manhattan']}

model3 = GridSearchCV(KNeighborsClassifier(), params, cv = 5)
model3.fit(x_train_s2, y_train)
pred3 = model3.predict(x_val_s2)

print(accuracy_score(y_val, pred3))
print(confusion_matrix(y_val, pred3))
print(classification_report(y_val, pred3))

In [ ]:
# 성능 튜닝 결과 확인하기
model3.best_params_

* 튜닝 과정 중 실험한 결과를 저장하고 시각화하기

In [ ]:
result = pd.DataFrame(model3.cv_results_)
result.head()

In [ ]:
plt.figure(figsize = (8,6))
sns.lineplot(x = 'param_n_neighbors', y = 'mean_test_score',
             hue = 'param_metric', data = result, marker = '.')
plt.grid()
plt.show()

### (3) 실험3
* 아래 다루는 내용은 경험을 통해 얻은 내용이므로 공식화 할 수는 없습니다. 참조 자료로만 사용하세요.😊
* hyper parameter는 어떻게 범위를 줘야 할까?
    * 경험이 있어서 범위를 지정할 수 있다면 : Grid Search
    * 전혀 경험이 없어서 어떻게 줘야 할지 모른다면
        * step 1 : Random Search로 넓은 범위를 주고 수행(듬성 듬성)
        * step 2 : Grid Search로 범위를 좁혀서(촘촘하게)

In [ ]:
# step1
params1 = {'n_estimators':range(10,301,10), 'learning_rate': np.linspace(0.001, 0.5, 100)}

model4 = RandomizedSearchCV(LGBMClassifier(verbose=-1), params1, cv = 5, n_iter = 20)
model4.fit(x_train, y_train)

print(model4.best_params_)

In [ ]:
# step2
params2 = {'n_estimators':range(4,31,2), 'learning_rate': np.linspace(0.01, 0.2, 20)}
model5 = GridSearchCV(LGBMClassifier(verbose=-1), params2, cv = 5)
model5.fit(x_train, y_train)

print(model5.best_params_)

In [ ]:
pred5 = model5.predict(x_val)

print(accuracy_score(y_val, pred5))
print(confusion_matrix(y_val, pred5))
print(classification_report(y_val, pred5))